# KENV

<a href=mailto:fuodorov1998@gmail.com>V. Fedorov</a>, <a href=mailto:nikdanila@bk.ru>D. Nikiforov</a>, <a href=http://www.inp.nsk.su/~petrenko/>A. Petrenko</a>, (Novosibirsk, 2019)

## GUI

Interactive interface to kenv on bokeh.

## Create a beam and accelerator.

In [1]:
import kenv as kv

In [2]:
beam = kv.Beam(energy=2,
               current=2e3,
               radius=50e-3,
               angular=50e-3,
               normalized_emittans=1000e-6)

In [3]:
accelerator = kv.Accelerator(0, 5, 0.01)

In [4]:
accelerator.add_acc('Acc. 1', 2.7, -1, 'input/E_z(z).dat')

In [5]:
Solenoids = [ 
    [ 0.5000, 0.02, 'input/B_z(z).dat', 'Sol. 1'],
    [ 1.0000, 0.02, 'input/B_z(z).dat', 'Sol. 2'],
    [ 1.5000, 0.02, 'input/B_z(z).dat', 'Sol. 3'],
    [ 2.0000, 0.02, 'input/B_z(z).dat', 'Sol. 4'],
    [ 2.5000, 0.02, 'input/B_z(z).dat', 'Sol. 5'],
    [ 3.0000, 0.02, 'input/B_z(z).dat', 'Sol. 6'],
    [ 3.5000, 0.02, 'input/B_z(z).dat', 'Sol. 7'],
    [ 4.0000, 0.02, 'input/B_z(z).dat', 'Sol. 8'],
    [ 4.5000, 0.02, 'input/B_z(z).dat', 'Sol. 9'],
]

In [6]:
for   z0, B0, filename, name in Solenoids:
    accelerator.Bz_beamline[name] = kv.Element(z0, B0, filename, name)

In [7]:
accelerator.compile()

Accelerator compiled.


In [8]:
simulation = kv.Simulation(beam, accelerator)

In [9]:
simulation.track()

Tracking is completed.


## Graphic

### bokeh

In [10]:
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file
from bokeh.models import HoverTool,  ColumnDataSource, Slider, CustomJS, Select, TextInput, Legend, Whisker
from bokeh.layouts import gridplot, row


import warnings
warnings.filterwarnings('ignore')

output_notebook()

Loading BokehJS ...

In [11]:
def update_text_style(beam, text_font='times', text_font_style='normal', text_font_size='13pt'  ):
    beam.title.text_font = text_font
    beam.title.text_font_style = text_font_style
    beam.title.text_font_size = text_font_size
    
    beam.xaxis.axis_label_text_font = text_font
    beam.xaxis.axis_label_text_font_style = text_font_style
    beam.xaxis.axis_label_text_font_size = text_font_size
    
    beam.yaxis.axis_label_text_font = text_font
    beam.yaxis.axis_label_text_font_style = text_font_style
    beam.yaxis.axis_label_text_font_size = text_font_size

    beam.legend.title_text_font = text_font
    beam.legend.title_text_font_style = text_font_style
    beam.legend.title_text_font_size = text_font_size
    
    beam.legend.label_text_font = text_font
    beam.legend.label_text_font_style = text_font_style
    beam.legend.label_text_font_size = text_font_size
    
    beam.yaxis.major_label_text_font_size = '12pt'
    beam.xaxis.major_label_text_font_size = '12pt'
    
    beam.legend.click_policy="hide"

In [12]:
def get_beam_data(beam=beam, accelerator=accelerator):
    accelerator.compile()
    simulation=kv.Simulation(beam, accelerator)
    simulation.track()
    sigma_x = simulation.envelope_x
    sigma_y = simulation.envelope_y
    z_array = np.array_split(accelerator.parameter, 1)
    Bz = accelerator.Bz(accelerator.parameter)
    Bz_array =  np.array_split(Bz, 1)
    sigma_x_array = np.array_split(sigma_x*1e3, 1)
    sigma_y_array = np.array_split(sigma_y*1e3, 1)
    return dict(z_array=z_array,
                sigma_x_array=sigma_x_array,
                sigma_y_array=sigma_y_array,
                Bz_array=Bz_array,
               )

In [13]:
beam_source = ColumnDataSource(data=get_beam_data())

Accelerator compiled.
Tracking is completed.


In [14]:
def modify_doc(doc,
               beam=beam,
               accelerator=accelerator):

    hover_beam = HoverTool(
            tooltips=[("2σ", "$y{1f} mm"),
                     ("z","$x{0.1f} m")],
        #mode='vline'
        )
    
    hover_field = HoverTool(
            tooltips=[("Field","$y{0.01f} T"),
                      ("z","$x{0.1f} m"),
                     ],
        #mode='vline'
        )
    
    input_size_x = TextInput(title="Radius x [m]", value=str(beam.radius_x))
    input_size_y = TextInput(title="Radius y [m]", value=str(beam.radius_y))
    
    input_angular_x = TextInput(title="Angular x [mrad]", value=str(beam.angular_x))
    input_angular_y = TextInput(title="Angular y [mrad]", value=str(beam.angular_y)) 
    
    select_sol = Select(title="Solenoid", value="Sol. 1", options=[itm.name for itm in accelerator.Bz_beamline.values()])
    input_sol = TextInput(title="max_field [T]", value=str(accelerator.Bz_beamline[str(select_sol.value)].max_field))
  
    select_acc = Select(title="Accel", value="Acc. 1", options=[itm.name for itm in accelerator.Ez_beamline.values()])
    input_acc = TextInput(title="max_field [MV/m]", value=str(accelerator.Ez_beamline[str(select_acc.value)].max_field))
    
    def update_size(attrname, old, new, beam=beam, accelerator=accelerator):
        beam.radius_x=float(input_size_x.value)
        beam.radius_y=float(input_size_y.value)
        beam_source.data=get_beam_data(beam, accelerator)
    input_size_x.on_change('value', update_size)
    input_size_y.on_change('value', update_size)
    
    def update_angular(attrname, old, new, beam=beam, accelerator=accelerator):
        beam.angular_x=float(input_angular_x.value)
        beam.angular_y=float(input_angular_y.value)
        beam_source.data=get_beam_data(beam, accelerator)
    input_angular_x.on_change('value', update_angular)
    input_angular_y.on_change('value', update_angular)
    
    def update_select_sol(attrname, old, new, beam=beam, accelerator=accelerator):
        input_sol.value = str(accelerator.Bz_beamline[str(select_sol.value)].max_field)
    select_sol.on_change('value', update_select_sol)
    
    def update_sol(attrname, old, new, beam=beam, accelerator=accelerator):
        accelerator.Bz_beamline[str(select_sol.value)].max_field = float(input_sol.value)
        beam_source.data=get_beam_data(beam, accelerator)
    input_sol.on_change('value', update_sol)
    
    def update_select_acc(attrname, old, new, beam=beam, accelerator=accelerator):
        input_acc.value = str(accelerator.Ez_beamline[str(select_acc.value)].max_field)
    select_acc.on_change('value', update_select_acc)
    
    def update_acc(attrname, old, new, beam=beam, accelerator=accelerator):
        accelerator.Ez_beamline[str(select_acc.value)].max_field = float(input_acc.value)
        print(accelerator)
        beam_source.data=get_beam_data(beam, accelerator)
    input_acc.on_change('value', update_acc)
    
    beam = figure(plot_width=800, plot_height=250, x_range=[accelerator.start, accelerator.stop], y_range=[0, 150], title="Beam envelope",
                 tools=[hover_beam,"pan,wheel_zoom,box_zoom,reset,save"],toolbar_location="right")
    
    beam.xaxis.axis_label = 'z, [m]'
    beam.yaxis.axis_label = 'Electron beam 2σ, [mm]'
    
    beam.legend.location = "top_left"
    
    beam.multi_line('z_array', 'sigma_x_array', source=beam_source,
                   legend='2σx',
                    line_color='red', line_alpha=0.5, line_width=3)
    beam.multi_line('z_array', 'sigma_y_array', source=beam_source,
                    legend='2σy', 
               line_color='blue', line_alpha=0.5, line_width=3)
    
    field = figure(plot_width=800, plot_height=200, x_range=beam.x_range, title='Field in accelerator',
                 tools=[hover_field,"pan,wheel_zoom,box_zoom,reset,save"],toolbar_location="right")
    
    field.multi_line('z_array', 'Bz_array', source=beam_source, line_color='blue', line_alpha=0.5, line_width=3)
   
    field.xaxis.axis_label = 'z, [m]'
    field.yaxis.axis_label = 'Bz'+' field, [T]'
    
    update_text_style(beam)
    update_text_style(field)
    
    layout = gridplot([[beam],
                       [field],
                       [row(select_sol, input_sol, input_size_x, input_size_y, width=800)],
                       [row(select_acc, input_acc, input_angular_x, input_angular_y, width=800)]])
    doc.add_root(layout)


In [15]:
show(modify_doc)